In [5]:
import os
import pandas as pd

if os.name == 'nt':
    os.environ["OMP_NUM_THREADS"] = '2'
import warnings

warnings.filterwarnings('ignore')
import sys

sys.path.append('../')
from utiliT.io import read_pickle, dump_pickle
from scripts.instance import *
from scripts.benders import *
import gc
from tqdm.notebook import tqdm, trange

In [3]:
file = "multipeak_200_200_400.pkl"
file_name = f"../data/{file}"
instance = read_pickle(file_name)
results = multi_cut(instance)
results['instance_name'] = file[:-4]
f"../results/run_results/{results['instance_name']}_{results['cut_method']}_{results['grouping_method']}_" \
f"{results['dr']}_{1}.pkl"

'../results/run_results/multipeak_200_200_400_multi_None_None_1.pkl'

In [4]:
file = "multipeak_100_100_100.pkl"
file_name = f"../data/{file}"
instance = read_pickle(file_name)
results = single_cut(instance)
results['instance_name'] = file[:-4]
f"../results/run_results/{results['instance_name']}_{results['cut_method']}_{results['grouping_method']}_" \
f"{results['dr']}_{1}.pkl"

'../results/run_results/multipeak_100_100_100_single_None_None_1.pkl'

In [5]:
file = "multipeak_200_200_400.pkl"
file_name = f"../data/{file}"
instance = read_pickle(file_name)
results = dropout_cut(instance, "affinity")
results['instance_name'] = file[:-4]
f"../results/run_results/{results['instance_name']}_{results['cut_method']}_{results['grouping_method']}_" \
f"{results['dr']}_{1}.pkl"

'../results/run_results/multipeak_200_200_400_dropout_affinity_False_1.pkl'

In [6]:
file = "multipeak_200_200_400.pkl"
file_name = f"../data/{file}"
instance = read_pickle(file_name)
results = hybrid(instance, "affinity")
results['instance_name'] = file[:-4]
f"../results/run_results/{results['instance_name']}_{results['cut_method']}_{results['grouping_method']}_" \
f"{results['dr']}_{1}.pkl"

'../results/run_results/multipeak_200_200_400_hybrid_affinity_False_1.pkl'

In [6]:
def runner(function, files=None, n_seeds=3):
    bad_instances = []
    if files is None:
        files = os.listdir("../data")
    for file in tqdm(files):
        for seed in range(n_seeds):
            instance_name = file[:-4]
            instance = read_pickle(f"../data/{file}")
            try:
                result = function(instance)
                result['instance_name'] = instance_name
                result['run_number'] = seed
                file_name = f"../results/run_results/{result['instance_name']}_{result['cut_method']}_{result['grouping_method']}_" \
                            f"{result['dr']}_{seed}.pkl"
                dump_pickle(result, file_name)
                del result
                gc.collect()
            except Exception as e:
                bad_instances.append(file)
                print(file)
                print(e)


In [7]:
def multi_runner(functions, files=None):
    print(f"Running {len(functions)} functions")
    i = 1
    for function in functions:
        print(f"Running function: {i}")
        i = i + 1
        runner(function, files=files)
        gc.collect()

In [8]:
def clean_results(terms):
    for term in terms:
        files = os.listdir("../results/run_results")
        deleting = [x for x in files if term in x]
        for f in deleting:
            os.remove(f"../results/run_results/{f}")

In [ ]:
functions = [
    lambda x: dropout_cut(x, "spectral"),  # Good
    lambda x: dropout_cut(x, "affinity"),  # Bad <- Rerun
    lambda x: dropout_cut(x, "hierarchical"),  # Good
    lambda x: dropout_cut(x, "kmeans"),  # Bad <- Rerun
    lambda x: dropout_cut(x, "random"),  # Bad Rerun
    multi_cut,
    single_cut
]
multi_runner(functions)

In [9]:
functions = [
    lambda x: dropout_cut(x, "affinity"),  # Bad <- Rerun
    lambda x: dropout_cut(x, "random"),  # Bad Rerun
]
multi_runner(functions)

Running 2 functions
Running function: 1


  0%|          | 0/96 [00:00<?, ?it/s]

Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-20
Running function: 2


  0%|          | 0/96 [00:00<?, ?it/s]

In [10]:
functions = [
    #lambda x: hybrid(x, "kmeans"),
    lambda x: hybrid(x, "affinity"),
    lambda x: hybrid(x, "hierarchical"),
    lambda x: hybrid(x, "spectral"),
    lambda x: hybrid(x, "random")
]
multi_runner(functions)

Running 4 functions
Running function: 1


  0%|          | 0/96 [00:00<?, ?it/s]

Running function: 2


  0%|          | 0/96 [00:00<?, ?it/s]

Running function: 3


  0%|          | 0/96 [00:00<?, ?it/s]

Running function: 4


  0%|          | 0/96 [00:00<?, ?it/s]

In [11]:
functions = [
    lambda x: dropout_cut(x, "spectral", dr=True),  # Good
    lambda x: dropout_cut(x, "affinity", dr=True),  # Bad <- Rerun
    lambda x: dropout_cut(x, "hierarchical", dr=True),  # Good
    lambda x: dropout_cut(x, "kmeans", dr=True),  # Bad <- Rerun
    lambda x: dropout_cut(x, "random", dr=True),  # Bad Rerun
]
multi_runner(functions)

Running 5 functions
Running function: 1


  0%|          | 0/96 [00:00<?, ?it/s]

Running function: 2


  0%|          | 0/96 [00:00<?, ?it/s]

Running function: 3


  0%|          | 0/96 [00:00<?, ?it/s]

Running function: 4


  0%|          | 0/96 [00:00<?, ?it/s]

Running function: 5


  0%|          | 0/96 [00:00<?, ?it/s]

In [12]:
functions = [
    #lambda x: hybrid(x, "kmeans"),
    lambda x: hybrid(x, "affinity", dr=True),
    lambda x: hybrid(x, "hierarchical",dr=True),
    lambda x: hybrid(x, "spectral",dr=True),
    lambda x: hybrid(x, "random",dr=True)
]
multi_runner(functions)

Running 4 functions
Running function: 1


  0%|          | 0/96 [00:00<?, ?it/s]

Running function: 2


  0%|          | 0/96 [00:00<?, ?it/s]

Running function: 3


  0%|          | 0/96 [00:00<?, ?it/s]

Running function: 4


  0%|          | 0/96 [00:00<?, ?it/s]